In [2]:
"""
BERTを用いたポジネガ分類器つくっちゃうよ

"""

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn

from transformers import BertJapaneseTokenizer, BertModel
from transformers import TrainingArguments, Trainer

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, precision_recall_fscore_support

from tqdm import tqdm
import glob, pickle

MODEL_NAME = "cl-tohoku/bert-base-japanese"
print(torch.cuda.is_available())

True


### 事前準備
学習データ成形、事前学習クラスのロード、BERTを使ったモデルのクラス作成

In [3]:
# ポジネガデータセット
df_dataset = pd.read_csv(
    'D:/DataSet/chABSA-dataset/chABSA-dataset/dataset.tsv',
    sep='\t', 
    header=None
).rename(columns={0:'text', 1:'label'}).loc[:, ['text', 'label']]

# ひとまずこういうFmtのデータに成形するところまでがんばる
df_dataset

,text,label
0,当社グループを取り巻く環境は、実質賃金が伸び悩むなか、消費者の皆様の生活防衛意識の高まりや節...,0
1,春から夏にかけましては個人消費の低迷などにより、きのこの価格は厳しい状況で推移いたしました,0
2,台湾の現地法人「台灣北斗生技股份有限公司」におきましては、ブランドの構築、企画提案などに力を...,0
3,化成品事業におきましては、引き続き厳しい販売環境にありましたが、中核である包装資材部門におき...,0
4,以上の結果、化成品事業の売上高は92億45百万円（同1.7％減）となりました,0
...,...,...
2808,当連結会計年度におきましては、連結子会社のデジタル・アドバタイジング・コンソーシアム株式会社...,1
2809,新規の自動ドアの売上台数は僅かに減少したものの、シートシャッターの大型物件に加え、取替の売上...,1
2810,"加えて、保守契約が堅調に増加し、売上高は6,952百万円（前年同期比1.2％増）となりました",1
2811,利益につきましては、取替工事の増加及び保守契約による安定的な利益の確保により、セグメント利益...,1


In [4]:
"""
https://dreamer-uma.com/pytorch-dataset/

対象タスクのデータを扱うDataset
データの格納と引き出し　DataLoaderと組み合わせてミニバッチ学習が可能
Datasetを自作する場合は必ず以下のメソッドを実装すること
__len__(): Datasetのサイズ（データ数）
__getitem__(): Datasetの要素にアクセス

"""
class PosiNegaDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        item = { k: torch.tensor(v[idx]) for k, v in self.encodings.items() }
        item["labels"] = torch.tensor(self.labels[idx]) # item["label"]でなくitem["labels"]が正しい
        return item

In [5]:
"""
BERTによる分類を行うレイヤのクラス

中に事前学習モデルを持ち、
input_ids -> model -> output -> classifier
という各レイヤのデータフローを流す
"""
class BertClassifier(nn.Module):
    def __init__(self, pretrained_model):
        super(BertClassifier, self).__init__()
        
        # 事前学習モデル
        self.bert = pretrained_model        
        # ドロップアウト層（過学習抑止効果）
        self.dropout = nn.Dropout(p=.1)
        # 線形変換層（全結合層）
        # ポジネガ（２カテゴリ）分類なので出力層は2
        self.classifier = nn.Linear(in_features=768, out_features=2)
        
        # 重み初期化
        nn.init.normal_(self.classifier.weight, std=0.02)
        nn.init.normal_(self.classifier.bias, 0)
    
    def forward(
        self, 
        input_ids, 
        labels, 
        attention_mask=None, token_type_ids=None, position_ids=None,
        head_mask=None, inputs_embeds=None, output_attentions=None,
        output_hidden_states=None, return_dict=None
    ):
        output = self.bert(
            input_ids=input_ids, 
            attention_mask=attention_mask,
            token_type_ids=token_type_ids, 
            position_ids=position_ids,
            head_mask=head_mask, 
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict
        )
        # トークン毎の特徴量（使わない）
        # last_hidden_state = output.last_hidden_state

        # 文代表（[CLS]トークン）の特徴量
        pooler_output = output.pooler_output
        pooler_output = self.dropout(pooler_output)
        # 分類タスク
        output_classifier = self.classifier(pooler_output)
        
        # loss計算
        loss_func = nn.CrossEntropyLoss()
        loss = loss_func(output_classifier.view(-1,2), labels.view(-1))
        
        # 必ず出力はlossが先
        return loss, output_classifier        
        

### 事前準備、以上
ここからは上で用意した各種クラスやモデルやデータセットを使ってタスクを解くコーディングをしていく

In [3]:
# トークナイザ
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

# 事前学習モデル
model = BertModel.from_pretrained(MODEL_NAME)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
# 特徴量X、ラベルyを取得
X, y = df_dataset["text"].values, df_dataset["label"].values

# train, val, test分割
# random_stateはシャッフルの乱数シード固定、stratifyは正例、負例のラベル数均一にする処理
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=0, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=0, stratify=y_val)

# トークナイザでモデルへのinputとなるようencodingする
max_len = 256 #512

enc_train = tokenizer(
    X_train.tolist(), 
    add_special_tokens=True, 
    max_length=max_len,
    padding='max_length',
    return_tensors='pt',
)
# tokenizer.convert_ids_to_tokens(enc_train['attention_masks'].tolist()[0])

enc_val = tokenizer(
    X_val.tolist(), 
    add_special_tokens=True, 
    max_length=max_len,
    padding='max_length',
    return_tensors='pt',
)

enc_test = tokenizer(
    X_test.tolist(), 
    add_special_tokens=True, 
    max_length=max_len,
    padding='max_length',
    return_tensors='pt',
)

In [8]:
# Datasetを作成
ds_train = PosiNegaDataset(enc_train, y_train)
ds_val = PosiNegaDataset(enc_val, y_val)
ds_test = PosiNegaDataset(enc_test, y_test)



In [9]:
ds_train.__getitem__(0)

C:\Users\licht\AppData\Local\Temp\ipykernel_2356\691864629.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = { k: torch.tensor(v[idx]) for k, v in self.encodings.items() }


{'input_ids': tensor([    2,   695,     5,   854,     6,   162,  5329,  6787,  1387,     5,
          6624,     7,  5217,  3913,    16,     9,     6,  2735,  1227,  1634,
           774,     9,     6,  1216,     5,  5330,    14,  7736,    15,    10,
          2442,     5,   881,    28,   130, 17144,    11,  6609,   251,  8025,
          3913,    10,    14,     6,  5580,  2040,  1634,   774,     9,     6,
          1216,  3279,     9,   837, 28913,     7,  7760,    15,    10,  1036,
           780,  3279,     5,  3641,   225,  6857, 28913,     7,  7760,    15,
          3913,    10,     3,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [10]:
# 自作BERTモデル
my_model = BertClassifier(model)

In [11]:
# 評価関数の設定
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [12]:
# Trainerを作成
training_args = TrainingArguments(
    output_dir='./outputs',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    no_cuda=False,
    evaluation_strategy='steps',
    eval_steps=50,
)

if "trainer" in locals():
    del trainer

trainer = Trainer(
    model=my_model,
    args=training_args,
    train_dataset=ds_train,
    eval_dataset=ds_val,
    compute_metrics=compute_metrics
)


In [13]:
# ファインチューニング
%time trainer.train()

C:\Users\licht\anaconda3\envs\py39\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1687
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 211
C:\Users\licht\AppData\Local\Temp\ipykernel_2356\691864629.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = { k: torch.tensor(v[idx]) for k, v in self.encodings.items() }


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,0.454600,0.308570,0.886323,0.883410,0.880631,0.887718
100,0.341400,0.248114,0.902309,0.899975,0.896832,0.905309
150,0.224900,0.222127,0.921847,0.918804,0.920053,0.917658
200,0.176700,0.218046,0.927176,0.924803,0.923499,0.926268


***** Running Evaluation *****
  Num examples = 563
  Batch size = 32
C:\Users\licht\AppData\Local\Temp\ipykernel_2356\691864629.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = { k: torch.tensor(v[idx]) for k, v in self.encodings.items() }
***** Running Evaluation *****
  Num examples = 563
  Batch size = 32
C:\Users\licht\AppData\Local\Temp\ipykernel_2356\691864629.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = { k: torch.tensor(v[idx]) for k, v in self.encodings.items() }
***** Running Evaluation *****
  Num examples = 563
  Batch size = 32
C:\Users\licht\AppData\Local\Temp\ipykernel_2356\691864629.py:20: UserWarning: To copy construct from a tensor, it is reco

TrainOutput(global_step=211, training_loss=0.42801548222794916, metrics={'train_runtime': 97.771, 'train_samples_per_second': 17.255, 'train_steps_per_second': 2.158, 'total_flos': 0.0, 'train_loss': 0.42801548222794916, 'epoch': 1.0})

In [14]:
# validationで評価
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 563
  Batch size = 32
C:\Users\licht\AppData\Local\Temp\ipykernel_2356\691864629.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = { k: torch.tensor(v[idx]) for k, v in self.encodings.items() }


{'eval_loss': 0.2222103774547577,
 'eval_accuracy': 0.9271758436944938,
 'eval_f1': 0.9247043856930831,
 'eval_precision': 0.9238853005521408,
 'eval_recall': 0.9255811521062678,
 'eval_runtime': 6.3374,
 'eval_samples_per_second': 88.837,
 'eval_steps_per_second': 2.84,
 'epoch': 1.0}

In [16]:
# testで評価
trainer.evaluate(ds_test)

***** Running Evaluation *****
  Num examples = 563
  Batch size = 32
C:\Users\licht\AppData\Local\Temp\ipykernel_2356\691864629.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = { k: torch.tensor(v[idx]) for k, v in self.encodings.items() }


{'eval_loss': 0.2615799307823181,
 'eval_accuracy': 0.9076376554174067,
 'eval_f1': 0.9045663172169196,
 'eval_precision': 0.9035505267264924,
 'eval_recall': 0.9056820856104385,
 'eval_runtime': 6.3069,
 'eval_samples_per_second': 89.267,
 'eval_steps_per_second': 2.854,
 'epoch': 1.0}

In [19]:
# モデル保存
torch.save(my_model.state_dict(), "./outputs/fine-tuned")

In [6]:
# 保存済みモデルを読み込む
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
new_model = BertClassifier(model).to(device)
new_model.load_state_dict(torch.load("./outputs/fine-tuned"))

<All keys matched successfully>

In [13]:
# つかってみる
sentences = [
    '当社の売り上げは１０年連続減少、経営赤字が続いている',
    '新製品の開発に成功、収益は過去最高を達成しました',
    'あの映画は本当に面白いからぜひ見てね',    
]

encodings = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
# ダミーでlabelsを追加
# loss計算には使われるが分類には無影響
encodings["labels"] = torch.tensor([0 for _ in range(len(sentences))])

encodings

{'input_ids': tensor([[    2, 12703,     5,  6376,     9,   121,    19,  1557,  2643,     6,
          1452,  9821,    14,  1822,    16,    33,     3,     0],
        [    2,   147,  2040,     5,   530,     7,  1320,     6,  9026,     9,
          2147,  1337,    11,  2485,    15,  3913,    10,     3],
        [    2,  7755,   450,     9, 10821, 17151,    40,  4403, 29065,   212,
            16,  1852,     3,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]]), 'labels': tensor([0, 0, 0])}

In [28]:
encodings.to(device)

with torch.no_grad():
    output = new_model(**encodings)

# output は return loss, output_classifier
# なので第２項が分類結果
output[1]

tensor([[ 2.7724, -1.9646],
        [-1.9792,  2.5945],
        [-0.1746,  0.8927]], device='cuda:0')

In [29]:
# 出力を分類スコアにするためにSoftmaxする
softmax_func = nn.Softmax(dim=1)
scores = softmax_func(output[1])

scores

tensor([[0.9913, 0.0087],
        [0.0102, 0.9898],
        [0.2559, 0.7441]], device='cuda:0')